<a href="https://colab.research.google.com/github/allfed/CropOpt/blob/standardize/scripts/creating_your_own_scenario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Intro
This notebook is supposed to show you how scenarios are created with this integrated model and show you to make your own. In this case here we want to run the model on a global scale and with no resilient foods. 


The first thing we have to do is to get the repository and install it. 

In [ ]:
import sys
import os
if 'google.colab' in sys.modules:
    # Get the repository and install everything if you are running this from a Colab notebook.
    !pip install git+https://github.com/allfed/allfed-integrated-model.git
    !pip install python-pptx
    !pip install pulp
else:
    # If you are running this locally you should install the dependencies in your own virtual environment.
    # change to the directory where the package is located and install it
    os.chdir("..")
    !pip install -e .

## Importing things
The model relies on a collection of external packages and internal classes that organize that code. Most important for us now are:
* Scenarios class: this is were all the parameters are stored and loaded for a given scenario
* ScenarioRunner class: this takes the newly created scenario and optimizes it 
* Plotter: plots the results of a scenario

In [ ]:
import numpy as np
from src.utilities.plotter import Plotter
from src.scenarios.run_scenario import ScenarioRunner
from src.scenarios.scenarios import Scenarios
import git
from pathlib import Path

repo_root = git.Repo(".", search_parent_directories=True).working_dir

## Filling the scenario with the fitting parameter values
We start by getting all the constant values from our scenario. This includes things like how many grass there is at the beginning of the simulation or what nutrition profile we want to use. The Scenarios class contains a wide variety of possible assumptions that can be used to create your own scenario. If you want to have your own, specific scenario assumptions you either have write them yourselve or create an issue in the repository of this model. 

In [ ]:
# This is needed to save all the constants
constants = {}
constants["CHECK_CONSTRAINTS"] = False
scenarios_loader = Scenarios()

constants_for_params = scenarios_loader.init_global_food_system_properties()

constants_for_params = scenarios_loader.get_no_resilient_food_scenario(
    constants_for_params
)

constants_for_params = scenarios_loader.set_excess_to_zero(constants_for_params)

constants_for_params = scenarios_loader.set_catastrophe_nutrition_profile(
    constants_for_params
)

constants_for_params = scenarios_loader.set_global_seasonality_nuclear_winter(
    constants_for_params
)
constants_for_params = scenarios_loader.set_global_grasses_nuclear_winter(
    constants_for_params
)
constants_for_params = scenarios_loader.set_stored_food_buffer_zero(
    constants_for_params
)
constants_for_params = scenarios_loader.set_efficient_feed_grazing_strategy(
    constants_for_params
)

constants_for_params = scenarios_loader.set_fish_nuclear_winter_reduction(
    constants_for_params
)
constants_for_params = scenarios_loader.include_protein(constants_for_params)

constants_for_params = scenarios_loader.include_fat(constants_for_params)
constants_for_params = scenarios_loader.cull_animals(constants_for_params)

constants_for_params = (
    scenarios_loader.set_nuclear_winter_global_disruption_to_crops(
        constants_for_params
    )
)
constants_for_params = scenarios_loader.set_immediate_shutoff(constants_for_params)

constants_for_params = scenarios_loader.set_waste_to_zero(constants_for_params)


## Optimizing the scenario
The main point of the integrated model is to assess how food sources have to be upscaled and distributed in time to make sure that there is enough food for everyone. To do this the model does a linear optimization of all available food sources. 

In [ ]:
    scenario_runner = ScenarioRunner()
    results = scenario_runner.run_and_analyze_scenario(
        constants_for_params, scenarios_loader
    )

    print("")
    print("")
    print("Estimated percet people fed, no resilient foods, no waste")
    print(results.percent_people_fed / 100 * 2100)
    print("")

    np.save(
        Path(repo_root) / "data" / "no_resilient_food_primary_results.npy",
        results,
        allow_pickle=True,
    )

This shows us that in the given configuration the ressources we have would easily feed everyone. Let's plot the results to see how the optimized food use looks like. 

In [ ]:
Plotter.plot_fig_1ab(
    results,
    77,
    "World",
    True,
    False,
    scenarios_loader.scenario_description,
)

We see that in that case outdoor crops and the later years meat would be most important for food production. We can also see that all macronutrients would be sufficient. 

Now try your own model :)

If you have any question contact morgan@allfed.info